In [3]:
pd.set_option("max_columns", None)

In [4]:
import xgboost


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  3 20:23:07 2019

@author: ericpicot
"""

"""
Created on Sun Mar  3 20:46:25 2019

@author: ericpicot
"""

####### Processing ----------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
#%autoreload 2
%load_ext autoreload
%autoreload 2
%aimport process
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import xgboost


def add_old_info(group, cols, _range):
    for i in _range:
        tmp_new_cols = [col+"diff_t-"+str(4) for col in cols]
        group[tmp_new_cols] = group[cols] - group[cols].shift(i).fillna(0)
        
        
    return group

######## Variables-----------------------------------------
w = 2
path = '/home/m420726/workspace/rugby/top_14_prediction/'
classement_original = pd.read_csv(path+"data_preparation/raw_data/ranking_results.csv", sep = "|")
journees = pd.read_csv(path+"data_preparation/raw_data/matches_results.csv", sep = "|")

print(journees.columns)
print(classement_original.columns)


# print(journees.shape, classement_original.shape)
# ############## --------------------------------------------
# to_window_compute = ['victoire','nul', 'defaite', 'bonus','ga']
journees = process.processing_journees(journees)
# classement_original = process.add_0ere_journee(classement_original)
# ####### Exploration ----------------------------------------

# classement_original = process.to_keep(classement_original)
# classement = process.add_journee_prec(classement_original)
# journees = process.to_keep(journees)
# journees = process.add_journee_prec(journees)

# classement["journee_int"] = classement["journee"].str[0:-11].astype(int)
# classement = (classement[classement["journee"]!= "0ere journee"]
# .groupby(["saison","equipe"],sort="journee_int")
# .apply(add_old_info,to_window_compute,[w])).drop("journee_int",axis = 1)

# data = process.return_data(classement, journees)

# #Pour l'instant, je ne prends pas en compte les matchs nuls:
# data["target"] = np.where(data["score_d"]>=data["score_e"],1,0) 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Index(['season_url', 'day_url', 'season_id', 'season_day', 'date', 'team_dom',
       'team_ext', 'score_dom', 'score_ext', 'bonus_dom', 'bonus_ext'],
      dtype='object')
Index(['season', 'day_url', 'day', 'classement', 'nb_pts', 'equipe',
       'nb_matchs_joues', 'victoire', 'nul', 'defaite', 'bonus', 'pts_marques',
       'pts_pris', 'ga'],
      dtype='object')


In [6]:
journees.head()

,season_url,day_url,season_id,season_day,date,team_dom,team_ext,score_dom,score_ext,bonus_dom,bonus_ext,day,day_number,month
0,/rugby-top-14/calendrier-resultats-rugby-top-1...,/rugby-top-14/calendrier-resultats-rugby-top-1...,2017-2018,1,Samedi 26 Août,Oyonnax,Toulouse,23,23,NaN,NaN,samedi,26,aout
1,/rugby-top-14/calendrier-resultats-rugby-top-1...,/rugby-top-14/calendrier-resultats-rugby-top-1...,2017-2018,1,Samedi 26 Août,Racing 92,Castres,25,21,NaN,Bd,samedi,26,aout
2,/rugby-top-14/calendrier-resultats-rugby-top-1...,/rugby-top-14/calendrier-resultats-rugby-top-1...,2017-2018,1,Samedi 26 Août,Montpellier,Agen,48,19,Bo,NaN,samedi,26,aout
3,/rugby-top-14/calendrier-resultats-rugby-top-1...,/rugby-top-14/calendrier-resultats-rugby-top-1...,2017-2018,1,Samedi 26 Août,Paris,Lyon,16,25,NaN,NaN,samedi,26,aout
4,/rugby-top-14/calendrier-resultats-rugby-top-1...,/rugby-top-14/calendrier-resultats-rugby-top-1...,2017-2018,1,Samedi 26 Août,Brive,La Rochelle,10,19,NaN,NaN,samedi,26,aout


In [42]:
classement_original.head()

,season,day_url,day,classement,nb_pts,equipe,nb_matchs_joues,victoire,nul,defaite,bonus,pts_marques,pts_pris,ga
0,2017-2018,/rugby-top-14/classement-rugby-top-14?season=2...,1,1.0,5.0,Montpellier Hérault Rugby,1.0,1.0,0.0,0.0,1.0,48.0,19.0,29.0
1,2017-2018,/rugby-top-14/classement-rugby-top-14?season=2...,1,2.0,4.0,RC Toulon,1.0,1.0,0.0,0.0,0.0,41.0,14.0,27.0
2,2017-2018,/rugby-top-14/classement-rugby-top-14?season=2...,1,3.0,4.0,LOU Rugby,1.0,1.0,0.0,0.0,0.0,25.0,16.0,9.0
3,2017-2018,/rugby-top-14/classement-rugby-top-14?season=2...,1,4.0,4.0,Stade Rochelais,1.0,1.0,0.0,0.0,0.0,19.0,10.0,9.0
4,2017-2018,/rugby-top-14/classement-rugby-top-14?season=2...,1,5.0,4.0,Union Bordeaux-Bègles,1.0,1.0,0.0,0.0,0.0,32.0,25.0,7.0


In [4]:

categorical_features = ['day', 'month']
data = process.get_dummies(data, categorical_features)




In [11]:

features = [
       'classement_d', 'nb_pts_d', 'nb_matchs_joues_d', 'victoire_d', 'nul_d',
       'defaite_d', 'pts_marques_d', 'pts_pris_d', 'ga_d',
       'classement_e', 'nb_pts_e', 'nb_matchs_joues_e', 'victoire_e', 'nul_e',
       'defaite_e', 'pts_marques_e', 'pts_pris_e', 'ga_e',
       'daydimanche', 'dayjeudi', 'daymercredi', 'daysamedi',
       'dayvendredi', 'monthaout', 'monthavril', 'monthdecembre',
       'monthfevrier', 'monthjanvier', 'monthjuin', 'monthmai', 'monthmars',
       'monthnovembre', 'monthoctobre', 'monthseptembre',
        'victoirediff_t-4d', 'nuldiff_t-4d',
       'defaitediff_t-4d', 'bonusdiff_t-4d', 'gadiff_t-4d',
       'victoirediff_t-4e',
       'nuldiff_t-4e', 'defaitediff_t-4e', 'bonusdiff_t-4e', 'gadiff_t-4e']

for f in features:
    data[f] = data[f].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(data[features+
                                                         ["equipe_d",
                                                          "equipe_e", 
                                                          "date", 
                                                          "saison"]], 
                                                    data["target"], 
                                                    test_size=.15, 
                                                    random_state=0)

In [16]:
list_tree = [10, 40,100,150]
list_max_depth = [4,6,8]
list_min_child_weight= [1, 2,5]
list_gamma = [1]
list_subsample = [1.0]
list_colsample_bytree = [1.0]

score = 0

for tree in list_tree:
    for max_depth in list_max_depth:
        for min_child_weight in list_min_child_weight:
            for gamma in list_gamma:
                for subsample in list_subsample:
                    for colsample_bytree in list_colsample_bytree:
                        model = xgboost.XGBClassifier(max_depth=max_depth, learning_rate=0.02,
                                                      n_estimators=tree, 
                                                      silent=True,
                                                      min_child_weight=min_child_weight, 
                                                      gamma=gamma,
                                                      subsample=subsample,
                                                      colsample_bytree=colsample_bytree, 
                                                      nthread=2,
                                                      reg_lambda=0,
                                                      reg_alpha=1)
                        model.fit(X_train[features], y_train)
                        score_ = model.score(X_test[features], y_test)
                        
                        if score_ > score:
                            print(score_)
                            tmp_tree = tree
                            tmp_max_depth = max_depth
                            tmp_min_child_weight = min_child_weight
                            tmp_gamma = gamma
                            tmp_subsample = subsample
                            tmp_colsample_bytree = colsample_bytree
                            score = score_ 
                            model_def = xgboost.XGBClassifier(max_depth=max_depth,
                                                              learning_rate=0.02,
                                                              n_estimators=tree, 
                                                              silent=True,
                                                              min_child_weight=min_child_weight, 
                                                              gamma=gamma,
                                                              subsample=subsample,
                                                              colsample_bytree=colsample_bytree, 
                                                              nthread=2,
                                                              reg_lambda=0,
                                                              reg_alpha=1)
                        model_def.fit(X_train[features], y_train)

print("results Grid Search : ")
print(tmp_tree)
print(tmp_max_depth)
print(tmp_min_child_weight)
print(tmp_gamma)
print(tmp_subsample)
print(tmp_colsample_bytree)
print("score :")
print(score)

0.7429718875502008
0.7469879518072289
0.7791164658634538
results Grid Search : 
40
4
1
1
1.0
1.0
score :
0.7791164658634538


In [17]:
model_def.score(X_test[features], y_test)

0.7791164658634538

In [35]:
pd.DataFrame([X_test["equipe_d"].values,
              X_test["equipe_e"].values,
              X_test["saison"].values,
              model_def.predict(X_test[features]),
              y_test.values],
              index = ["equipe_d",
                      "equipe_e",
                      "saison",
                      "prediction",
                      "reality"]).T

,equipe_d,equipe_e,saison,prediction,reality
0,La Rochelle,Paris,TOP 14 2014-2015,1,1
1,Toulon,Grenoble,TOP 14 2016-2017,1,1
2,Toulouse,Clermont,TOP 14 Orange 2011-2012,1,1
3,Brive,Bayonne,TOP 14 Orange 2010-2011,1,0
4,Toulouse,Paris,TOP 14 2015-2016,1,1
5,Agen,Bayonne,TOP 14 Orange 2011-2012,1,1
6,Toulouse,Toulon,TOP 14 2016-2017,1,0
7,Bordeaux-Bègles,Oyonnax,TOP 14 2017-2018,1,0
8,Castres,Toulon,TOP 14 2017-2018,1,1
9,Perpignan,Mont-de-Marsan,TOP 14 2012-2013,1,1


In [20]:
X_test.head(10)

,classement_d,nb_pts_d,nb_matchs_joues_d,victoire_d,nul_d,defaite_d,pts_marques_d,pts_pris_d,ga_d,classement_e,nb_pts_e,nb_matchs_joues_e,victoire_e,nul_e,defaite_e,pts_marques_e,pts_pris_e,ga_e,daydimanche,dayjeudi,daymercredi,daysamedi,dayvendredi,monthaout,monthavril,monthdecembre,monthfevrier,monthjanvier,monthjuin,monthmai,monthmars,monthnovembre,monthoctobre,monthseptembre,victoirediff_t-4d,nuldiff_t-4d,defaitediff_t-4d,bonusdiff_t-4d,gadiff_t-4d,victoirediff_t-4e,nuldiff_t-4e,defaitediff_t-4e,bonusdiff_t-4e,gadiff_t-4e,equipe_d,equipe_e,date,saison
1002,10,44,21,9,2,10,423,532,0.0,3,59,21,13,0,8,497,469,28.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,La Rochelle,Paris,Vendredi 10 Avril,TOP 14 2014-2015
1257,4,22,8,4,1,3,196,171,25.0,13,11,8,2,0,6,182,263,81.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1.0,1.0,0.0,1.0,22.0,1.0,0.0,1.0,1.0,0.0,Toulon,Grenoble,Samedi 29 Octobre,TOP 14 2016-2017
379,3,18,5,4,0,1,121,86,35.0,1,21,5,5,0,0,114,43,71.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2.0,0.0,0.0,1.0,25.0,2.0,0.0,0.0,0.0,17.0,Toulouse,Clermont,Samedi 01 Octobre,TOP 14 Orange 2011-2012
278,12,26,16,4,2,10,266,353,0.0,8,40,16,9,0,7,320,311,9.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,-15.0,Brive,Bayonne,Jeudi 27 Janvier,TOP 14 Orange 2010-2011
1154,6,48,18,9,2,7,457,301,156.0,12,31,18,7,0,11,356,408,52.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,Toulouse,Paris,Dimanche 27 Mars,TOP 14 2015-2016
414,6,28,11,6,1,4,210,215,0.0,11,20,11,4,1,6,173,220,0.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,Agen,Bayonne,Dimanche 04 Décembre,TOP 14 Orange 2011-2012
1224,5,8,3,2,0,1,56,57,1.0,11,6,3,1,0,2,66,71,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,1.0,0.0,-7.0,1.0,0.0,1.0,1.0,0.0,Toulouse,Toulon,Dimanche 11 Septembre,TOP 14 2016-2017
1498,9,39,19,8,1,10,445,444,1.0,14,22,19,3,3,13,396,620,224.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,2.0,1.0,-7.0,2.0,0.0,0.0,0.0,-12.0,Bordeaux-Bègles,Oyonnax,Samedi 10 Mars,TOP 14 2017-2018
1432,7,25,10,5,0,5,253,237,16.0,8,25,10,5,0,5,263,249,14.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,-13.0,Castres,Toulon,Samedi 25 Novembre,TOP 14 2017-2018
554,9,11,6,2,0,4,144,167,0.0,14,0,6,0,0,6,76,195,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,Perpignan,Mont-de-Marsan,Samedi 29 Septembre,TOP 14 2012-2013
